# Environment Setup


In [ ]:
#@markdown <h3>⬅️ Click Here to START server</h3>
#@markdown <center><h3>SimpleTorrent is a a self-hosted remote torrent client.</h3></center><br>
#@markdown <center><h5>TBD add description here!</h5></center><br>
import os
import time
import pathlib 
import urllib.request


!pip install dill
!pip install ffpb
from IPython.display import clear_output
HOME = os.path.expanduser("~")
py_lib="my_lib"
if not os.path.exists(f"{HOME}/.ipython/{py_lib}.py"):
    hCode = f"https://raw.githubusercontent.com/ccyang2019/" \
                f"Google-Colab-Torrent/master/res/{py_lib}.py"
    print(f"url={hCode}")
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/{py_lib}.py")
from my_lib import (
  findProcess,   
  runSh
)
# Install colab_ssh on google colab
#@title Keep parameter in default if you don't know how to change 
!pip install colab_ssh --upgrade
YOUR_SSH_PASSWORD="x12345" #@param {type:"string"}
YOUR_ngrok_TOKEN="1ghsYrC5aJyGsKuSphRkPq6cERD_3enXbp6hznhESxjKqq1We" #@param {type:"string"}
from colab_ssh import launch_ssh, init_git, get_tunnel_config 
if not findProcess("ngrok"):
  ret=launch_ssh(YOUR_ngrok_TOKEN, YOUR_SSH_PASSWORD)
  ret = get_tunnel_config()
  print(f"[MSG] open ngrok ssh: ssh -p {ret['port']} root@{ret['domain']}, password={YOUR_SSH_PASSWORD}")
else:
  print(f"ngrok is running")
  #print(f"{colab_ssh.get_tunnel_config()}")
  ret = get_tunnel_config()
  print(f"{ret}")
  print(f"[MSG] open ngrok ssh: ssh -p {ret['port']} root@{ret['domain']}, password={YOUR_SSH_PASSWORD}")

YOUR_GITHUB_USERNAME="ccyang2019" #@param {type:"string"}
YOUR_GITHUB_REPOS="colab-utilities" #@param {type:"string"}
YOUR_GITHUB_PERSONAL_TOKEN="c0fd460a493976b5ebd726424d201d4d3569ab2e" #@param {type:"string"}
YOUR_GITHUB_BRANCH="master" #@param {type:"string"}
YOUR_GITHUB_EMAIL="ccyang2015.79g@g2.nctu.edu.tw" #@param {type:"string"}

init_git(f"https://github.com/{YOUR_GITHUB_USERNAME}/{YOUR_GITHUB_REPOS}.git",
         personal_token=f"{YOUR_GITHUB_PERSONAL_TOKEN}", 
         branch=f"{YOUR_GITHUB_BRANCH}",
         email=f"{YOUR_GITHUB_EMAIL}",
         username=f"{YOUR_GITHUB_USERNAME}")

url=https://raw.githubusercontent.com/ccyang2019/Google-Colab-Torrent/master/res/my_lib.py
Successfully running 0.tcp.ngrok.io:10073
[Optional] You can also connect with VSCode SSH Remote extension using this configuration:

	Host google_colab_ssh
		HostName 0.tcp.ngrok.io
		User root
		Port 10073
	  
[MSG] open ngrok ssh: ssh -p 10073 root@0.tcp.ngrok.io, password=x12345
Successfully cloned the repository


In [ ]:
from my_lib import (
  runSh,
  findProcess,   
)
colab_ut_path = "/content/colab-utilities"
print("Start to Mount Grive ...")
runSh(f"{colab_ut_path}/mount_gdrive.py")
print("Start to Install FFMPEG ...")
runSh(f"{colab_ut_path}/install_ffmpeg.py")

Start to Mount Grive ...
Start to Install FFMPEG ...


0

# Function Definition#

In [ ]:
import logging,sys,os
logger = logging.getLogger(__name__)

## Func: run_video_convert

In [ ]:
def run_video_convert(video):
#conv = ['AAC', '4Kto1080']
  GPU_EN=1
  USE_FFPB=1
  #TEST=0
  conv=video['conv']
  opt_a = []
  if USE_FFPB:
    opt_a.append("ffpb -y")
  else:
    opt_a.append("ffmpeg -y")

  map_a=[]
  src= os.path.join(video['dir'], video['base']+video['ext'])
  opt_a.append("-i " + src)
  map_a.append("-map 0:v?")
  map_a.append("-map 0:a?")
  map_a.append("-map 0:s?")


  new_sub_idx = !ffprobe {src}  -show_streams -select_streams s 2>&1 | grep 'codec_type=subtitle'  | wc -l
  print(f"new={new_sub_idx}")
  new_sub_idx = int(new_sub_idx[0])
  num=1
  # subtitle
  for s in video['subs']:
    opt_a.append(f"-i \'{s}\'")
    bn = os.path.basename(s)
    bn = re.sub(rf"^{video['base']}\.(.*)\.(ass|srt)$", r"\1", bn, flags=re.IGNORECASE)
    map_a.append(f"-map {num}")
    if bn != "":
      map_a.append(f"-metadata:s:s:{new_sub_idx} language='{bn}'")
    num = num + 1
    new_sub_idx = new_sub_idx+1

  if TEST_CONVERT:
    opt_a.append("-ss 00:00:00 -t 00:01:00")
    
  opt_a.append("-movflags use_metadata_tag")
  opt_a.append("-ignore_unknown")
  opt_a.append(" ".join(map_a))
  
  if GPU_EN:
    venc = "-c:v hevc_nvenc -cq 35 -preset slower"
  else:
    venc = "-c:v libx265 -crf 35 -preset slow"

  if "4Kto1080" in conv:
    opt_a.append(f"-vf scale=1920:1080 {venc}")
  #else:
    #opt_a.append("-c:v copy")

  if "X265" in conv:
    opt_a.append(venc)
  else:
    opt_a.append("-c:v copy")

  if "AAC" in conv:
    opt_a.append("-c:a aac")
  else:
    opt_a.append("-c:a copy")

  # subtitle
  # if not add, will all convert to ass
  opt_a.append("-c:s copy")


  dst = os.path.join(video['dir'], video['new_base']+".mkv")
  opt_a.append(dst)
  cmd = " ".join(opt_a)
  # -ac 6 -ar 48000 -ab 400k -c:a libfdk_aac" ...=> only support at ffmpeg 4.x
  print(f"cmd={cmd}")
  #ret=runSh(cmd, shell=True,output=True)
  #print(f"Ret={ret}")
  !{cmd}

## Func: get_video_files

In [ ]:
def get_video_files(src_path):
  import glob
  video_ext = [".m4v", ".mp4", ".mkv", ".ts"]
  videos = []
  for _root, _dirs, _files in os.walk(src_path, followlinks=True):
          for _file in _files:
              print(_file)
              ext = os.path.splitext(_file)[-1]
              base= os.path.splitext(_file)[-2]
              vfile = os.path.join(_root, _file)
              if ext.lower() in video_ext:
                v_d = {"base": base, 
                      "dir" : _root, 
                      "ext" : ext, 
                      'subs' : [],
                      'audio' : '',
                      'video' : '',
                      'height' : '',
                       'width' : '',
                      }
                [v_d['video'], v_d['width'], v_d['height']]=runSh(f"ffprobe -v error  -select_streams v:0 -show_entries stream=codec_name,width,height -of csv=s=-:p=0 {vfile}",output=True,shell=True).split("-")
                v_d['audio']=runSh(f"ffprobe -v error  -select_streams a:0 -show_entries stream=codec_name -of csv=s=-:p=0 {vfile}",output=True,shell=True)
                for type in ["srt","ass"]:
                  pat = os.path.join(_root, f"{base}*.{type}")
                  for sub in glob.glob(pat):
                    v_d['subs'].append(sub)
                logger.debug(v_d)
                videos.append(v_d)                                  
              #print(f"{ext} {base}")

  print(f"Number of vidoes = {len(videos)}")             #find video subs
  return videos

## Func: check_convert_request

In [ ]:
def check_convert_request(video,convert):
  import glob
  En_4kto1080=False
  _file=video['base']
  _new_file=_file
  conv=[]
  if "4Kto1080" in convert and video['width']=='2160':
    En_4kto1080=True
    conv.append("4Kto1080")
    match = re.search(r'(4K|2160P)', _file,flags=re.IGNORECASE)
    if match:
      _new_file = re.sub(match.group(1), '1080P', _file, flags=re.IGNORECASE)
    else:
      _new_file = f"{_file}.1080P"

  if "X265" in convert and video['video']!='hevc':
    conv.append("X265")
    match = re.search(r'([HX]264)', _file,flags=re.IGNORECASE)
    if match:
      _new_file = re.sub(match.group(1), 'X265', _file, flags=re.IGNORECASE)
    else:
      _new_file = f"{_file}.X265"
    
  En_AAC=False
  if "AAC" in convert and video['audio']!="aac":
    En_AAC=True
    conv.append("AAC")
    match = re.search(rf"{video['audio']}", _new_file,flags=re.IGNORECASE)
    if match:
      _new_file = re.sub(match.group(1), 'AAC', _new_file, flags=re.IGNORECASE)
    else:
      _new_file = f"{_new_file}.AAC"
  # 
  if _new_file == _file:
    return [False,f"SAME",conv]
  # check if _new_file existed? 
  pat = os.path.join(video['dir'],f"{_new_file}*")
  video_ext = [".m4v", ".mp4", ".mkv", ".ts"]
  if not IGNORE_EXISTED:
    for _f in glob.glob(pat):
      ext = os.path.splitext(_f)[-1]
      if ext.lower() in video_ext:
        return [False,f"Existed {_new_file}",conv]
  # check if _file locked to convert?
  if not IGNORE_LOCK:
    pat = os.path.join(video['dir'],f"lock-{_file}*")
    for _f in glob.glob(pat):
        return [False,f"Locked {_file}",conv]
    # lock it
  lock_file = os.path.join(video['dir'], f"lock-{_file}")
  runSh(f"echo `date` > {lock_file}", shell=True)
  
  return [True, _new_file, conv]

# Video Convert

In [102]:
colab_ut_path = "/content/colab-utilities"
#runSh(f"{colab_ut_path}/mount_gdrive.py")
#runSh(f"{colab_ut_path}/install_ffmpeg.py")
#TEST_CONVERT = True  #@param {type:"boolean"}
IGNORE_LOCK = True  #@param {type:"boolean"}
GDRIVE_FOLDER_ID =  "11rVFYUYVrh9nnLnRrLNvLBc8Duwciyie" #@param {type:"string"}
PATH_GDRIVE_MOUNT =  "/content/GDRIVE" #@param {type:"string"}
VIDEO_PATH =  "" #@param {type:"string"}
os.environ['GDRIVE_FOLDER_ID'] = f"{GDRIVE_FOLDER_ID}" 
os.environ['PATH_GDRIVE_MOUNT'] = f"{PATH_GDRIVE_MOUNT}"
os.environ['VIDEO_PATH'] = f"{VIDEO_PATH}"
V_ACC = True  #@param {type:"boolean"}
V_4Kto1080 = True  #@param {type:"boolean"}
V_X265 = True  #@param {type:"boolean"}
#runSh(f"{colab_ut_path}/mount_gdrive.py", output=True, shell=True)
!{colab_ut_path}/mount_gdrive.py
print("Start to Install FFMPEG ...")
runSh(f"{colab_ut_path}/install_ffmpeg.py")


Run: /content/colab-utilities/mount_gdrive.py , dir=/content/colab-utilities
Use system env variable values
nohup: ignoring input and appending output to 'nohup.out'
Start to Install FFMPEG ...


0

In [ ]:
#clear_output()
TEST_CONVERT = False  #@param {type:"boolean"}
IGNORE_EXISTED = True  #@param {type:"boolean"}

print(f"VIDEO SRC DIR={PATH_GDRIVE_MOUNT}")
videos= get_video_files(os.path.join(PATH_GDRIVE_MOUNT))
print(videos)
#videos= get_video_files("/content/GDRIVE")
conv=[]
for opt in ["V_4Kto1080","V_AAC", "V_X265"]:
  if eval("opt"):
    conv.append(re.sub(r'V_','', opt))

print(f"CONV={conv}")
for video in videos:
  print(f"video={video}")
  [run, _new_file, conv] = check_convert_request(video, convert)
  video['new_base'] = _new_file
  video['conv'] = conv
  if run:
    print(f"Run={run}, New File = {_new_file}, conv={conv}")
    try:
      run_video_convert(video)
      lock_file = os.path.join(video['dir'], f"lock-{video['base']}")
      !rm {lock_file}
    except:
      logger.ERROR(f"Fail to convert : {video['base']}")


#with open("notebook_var.db", "wb") as dill_file:
#  dill.dump(videos, dill_file)
# ffprobe Fleabag.S01E01.1080p.BluRay.x264-SHORTBREHD.mkv  -show_streams -select_streams s 2>&1 | grep 'codec_type=subtitle'  | wc -l


VIDEO SRC DIR=/content/GDRIVE
德云斗笑社.E01.2020.WEB-DL.1080P.H264.mp4
德云斗笑社.E02.2020.WEB-DL.1080P.H264.mp4
Number of vidoes = 2
[{'base': '德云斗笑社.E01.2020.WEB-DL.1080P.H264', 'dir': '/content/GDRIVE/S01', 'ext': '.mp4', 'subs': [], 'audio': 'aac', 'video': 'h264', 'height': '1080', 'width': '1920'}, {'base': '德云斗笑社.E02.2020.WEB-DL.1080P.H264', 'dir': '/content/GDRIVE/S01', 'ext': '.mp4', 'subs': [], 'audio': 'aac', 'video': 'h264', 'height': '1080', 'width': '1920'}]
CONV=['4Kto1080', 'AAC', 'X265']
video={'base': '德云斗笑社.E01.2020.WEB-DL.1080P.H264', 'dir': '/content/GDRIVE/S01', 'ext': '.mp4', 'subs': [], 'audio': 'aac', 'video': 'h264', 'height': '1080', 'width': '1920'}
Run=True, New File = 德云斗笑社.E01.2020.WEB-DL.1080P.X265, conv=['X265']
new=['0']
cmd=ffpb -y -i /content/GDRIVE/S01/德云斗笑社.E01.2020.WEB-DL.1080P.H264.mp4 -movflags use_metadata_tag -ignore_unknown -map 0:v? -map 0:a? -map 0:s? -c:v hevc_nvenc -cq 35 -preset slow -c:a copy -c:s copy /content/GDRIVE/S01/德云斗笑社.E01.2020.WEB-DL.1

# New Section

In [ ]:

#with open("notebook_var.db", "rb") as dill_file:
#  videos= dill.load(dill_file)
video=videos[0]
#bn = re.sub(rf"^{video['base']}\.(.*)\.[ass|srt]$", \1, video['subs'][0], flags=re.IGNORECASE)
bn = os.path.basename(video['subs'][0])
print(video['base'])
bn = re.sub(rf"^{video['base']}\.(.*)\.(ass|srt)$", r"\1", bn, flags=re.IGNORECASE)
print(bn)

Fleabag.S01E01.1080p.BluRay.x264-SHORTBREHD
chs&eng
